In [41]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pymongo

# tool for browsing with Spliter
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser

In [42]:
# Initialize Pymongo to work with mongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

### Scrape for latest new title and text paragraph

In [89]:
news_url = 'https://mars.nasa.gov/news/'

response = requests.get(news_url)
# creat bs object
soup = bs(response.text, 'html.parser')

In [44]:
title_result = soup.find('div', class_='content_title')
news_title = title_result.text

In [45]:
teaser_result = soup.find('div', class_='rollover_description_inner')
teaser_para = teaser_result.text

### JPL Mars image--feature image

In [46]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/kaipham/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [92]:
# browse the link 
img_url = 'https://spaceimages-mars.com/'
browser.visit(img_url)

In [93]:
html = browser.html
soup = bs(html, 'html.parser')

print(soup.prettify())

<html class="">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
  <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <title>
   Space Image
  </title>
  <style type="text/css">
   .fancybox-margin{margin-right:0px;}
  </style>
 </head>
 <body>
  <div class="header">
   <nav class="navbar navbar-expand-lg">
    <a class="navbar-brand" href="#">
     <img id="logo" src="image/nasa.png"/>
     <span class="logo">
      Jet Propulsion Laboratory
     </span>
     <span class="logo1">
      California Institute of Technology
     </span>
    </a>
    <button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle 

In [94]:
# click on the buttone "Full image"
# when it clicked the variable <html> also move to the new page, because the action if apply onto <browser>
browser.links.find_by_partial_text('FULL IMAGE').click()

In [78]:
soup.find('img', class_='fancybox-image')['src']

'image/featured/mars3.jpg'

In [76]:
# Store the image url into a variable
feature_image_url = f"{img_url}{soup.find('img', class_='fancybox-image')['src']}"

### Mars Facts with Pandas Scraping (read_html)

In [80]:
fact_url = 'https://galaxyfacts-mars.com/'

In [81]:
tables = pd.read_html(fact_url)

In [88]:
mars_facts_df = tables[1]
mars_facts_df.columns = ["Entities", "Info"]

,Entities,Info
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
